In [1]:
# built-in imports
import sys
import os
import re
import json
import gzip
import pickle
import array

# third-party imports
import pandas as pd
import numpy as np
import scipy.sparse as ssp
import dgl
import torch
import torchtext

# local imports
sys.path.insert(0, '../src/pinsage')
from builder import PandasGraphBuilder
from data_utils import *

Using backend: pytorch


In [41]:
# get directory of data files
directory = '../data'
# get output file path
output_path = './processed-amazon.pkl'

In [42]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## Review Data

In [43]:
reviews_path = os.path.join(directory, 'reviews_Electronics_5.json.gz')
reviews = getDF(reviews_path)
reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [44]:
reviews.shape

(1689188, 9)

## Clean Reviews Data

In [45]:
reviews = reviews.drop(['reviewerName', 'reviewTime'], axis = 1).dropna()
reviews.head()

,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime
0,AO94DHGC771SJ,0528881469,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200
1,AMO214LNFCEI4,0528881469,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200
2,A3N7T0DY83Y4IG,0528881469,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400
3,A1H8PY3QHMQQA0,0528881469,"[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800
4,A24EV6RXELQZ63,0528881469,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400


In [57]:
reviews['helpful'].apply(lambda x: x[0]/x[1] if x[1] != 0 else 0.0)

0          0.000000
1          0.800000
2          0.955556
3          0.900000
4          0.000000
             ...   
1689183    1.000000
1689184    0.000000
1689185    0.000000
1689186    0.000000
1689187    0.000000
Name: helpful, Length: 1689188, dtype: float64

## Users

In [67]:
users = reviews[['reviewerID']].drop_duplicates()
users['reviewerID'].values

array(['AO94DHGC771SJ', 'AMO214LNFCEI4', 'A3N7T0DY83Y4IG', ...,
       'A2P7TRORW0S26C', 'A1KQNP8MOJDJKC', 'A1FQ3HRVXA4A5B'], dtype=object)

## Products

In [9]:
products_path = os.path.join(directory, 'meta_Electronics.json.gz')
products = getDF(products_path)
products.head()

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [10]:
print(products.shape)
print(products.dtypes)

(498196, 9)
asin            object
imUrl           object
description     object
categories      object
title           object
price          float64
salesRank       object
related         object
brand           object
dtype: object


In [61]:
users

,reviewerID
0,AO94DHGC771SJ
1,AMO214LNFCEI4
2,A3N7T0DY83Y4IG
3,A1H8PY3QHMQQA0
4,A24EV6RXELQZ63
...,...
1649261,A35YETXEOM4T88
1649271,A2W6U6RIH05A5K
1649278,A2P7TRORW0S26C
1664617,A1KQNP8MOJDJKC


In [62]:
# Filter the users and items that never appear in the rating table.
distinct_users_in_reviews = reviews['reviewerID'].unique()
users = users.copy()[users['reviewerID'].isin(distinct_users_in_reviews)]
with open(os.path.join(directory, 'users.out'), 'wb') as fp:
    pickle.dump(distinct_users_in_reviews, fp)
users

,reviewerID
0,AO94DHGC771SJ
1,AMO214LNFCEI4
2,A3N7T0DY83Y4IG
3,A1H8PY3QHMQQA0
4,A24EV6RXELQZ63
...,...
1649261,A35YETXEOM4T88
1649271,A2W6U6RIH05A5K
1649278,A2P7TRORW0S26C
1664617,A1KQNP8MOJDJKC


In [20]:
distinct_products_in_reviews = reviews['asin'].unique()
products = products.copy()[products['asin'].isin(distinct_products_in_reviews)]
with open(os.path.join(directory, 'products.out'), 'wb') as fp:
    pickle.dump(distinct_products_in_reviews, fp)
products

,asin,imUrl,description,categories,title,price,salesRank,related,brand
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
15,0594451647,http://ecx.images-amazon.com/images/I/51RjSETO...,HDTV Adapter Kit for NOOK HD and NOOK HD+\nThi...,"[[Electronics, Computers & Accessories, Touch ...",Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,49.95,NaN,"{'also_bought': ['B009L7EEZA', 'B00AGAYQEU', '...",NaN
20,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[[Electronics, eBook Readers & Accessories, Po...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,19.65,NaN,"{'also_bought': ['B00AAKLIIS', 'B00A668GUO', '...",Barnes &amp; Noble
38,0972683275,http://ecx.images-amazon.com/images/I/41hYJ9Mw...,The VideoSecu TV mount is a mounting solution ...,"[[Electronics, Accessories & Supplies, Audio &...",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,29.99,{},"{'also_bought': ['B000X3KOD2', 'B0074FGR74', '...",VideoSecu
53,1400532620,http://ecx.images-amazon.com/images/I/519ca3cu...,Barnes & Noble Nook eReader - no 3GMeet nook. ...,"[[Electronics, eBook Readers & Accessories]]",Barnes &amp; Noble Nook eReader - no 3G,74.95,{'Electronics': 23071},"{'also_bought': ['B0035CLBT4', 'B004X18N24', '...",Barnes &amp; Noble
...,...,...,...,...,...,...,...,...,...
497900,B00L2442H0,http://ecx.images-amazon.com/images/I/51NFBdDZ...,Description:Add up to 4 peripherals quickly an...,"[[Electronics, Computers & Accessories, Networ...",Sabrent 4 Port Portable USB 2.0 Hub (9.5&quot;...,5.99,{'Electronics': 1383},"{'also_bought': ['B00IRV2DL8', 'B00GU8OIYA', '...",Sabrent
497904,B00L26YDA4,http://ecx.images-amazon.com/images/I/41061q4C...,Description:The Sabrent Wifi Receiver lets you...,"[[Electronics, Car & Vehicle Electronics, Vehi...",Sabrent Wifi Audio Receiver (Supports DLNA and...,35.99,{'Cell Phones & Accessories': 6139},"{'also_bought': ['B00KB5QEYK', 'B00L2JQ8AE', '...",Sabrent
497905,B00L21HC7A,http://ecx.images-amazon.com/images/I/513kT0it...,Description:The Sabrent CR-CCU3 3-Slot Card Re...,"[[Electronics, Computers & Accessories, Cables...",Sabrent USB 3.0 SuperSpeed 3 slot Memory Card ...,14.99,{},"{'also_viewed': ['B00GAKX34E', 'B00L2442H0', '...",Sabrent
497926,B00L3YHF6O,http://ecx.images-amazon.com/images/I/41SBx7QY...,"Mind-Shattering Performance, Precision-Tuned F...","[[Electronics, Home Audio, Stereo Components, ...",NEW! Creative Sound Blaster Roar: Portable NFC...,149.99,{'Cell Phones & Accessories': 131},"{'also_bought': ['B00LBNW2TC', 'B00L8I6SFY', '...",NaN


## Events

In [13]:
events = reviews[['reviewerID', 'asin', 'unixReviewTime']]
events

,reviewerID,asin,unixReviewTime
0,AO94DHGC771SJ,0528881469,1370131200
1,AMO214LNFCEI4,0528881469,1290643200
2,A3N7T0DY83Y4IG,0528881469,1283990400
3,A1H8PY3QHMQQA0,0528881469,1290556800
4,A24EV6RXELQZ63,0528881469,1317254400
...,...,...,...
1689183,A34BZM6S9L7QI4,B00LGQ6HL8,1405555200
1689184,A1G650TTTHEAL5,B00LGQ6HL8,1405382400
1689185,A25C2M3QF9G7OQ,B00LGQ6HL8,1405555200
1689186,A1E1LEVQ9VQNK,B00LGQ6HL8,1405641600


## Image Features

In [14]:
t = {1: ['ASIN1', [1.0, 2.0, 3.0]],
 2: ['ASIN2', [-1.0, -2.0, -3.0]]}

In [21]:
def readImageFeatures(path):
    f = open(path, 'rb')
    while True:
        asin = str(f.read(10), 'utf-8')
        if asin == '': 
            print('breaking...')
            break
        a = array.array('f')
        a.fromfile(f, 4096)
        yield asin, a.tolist()

In [22]:
# get directory of data files
data_dir = '../data'

image_fn = 'image_features_Electronics.b'
image_features_path = os.path.join(data_dir, image_fn)

product_list_fn = 'products.out'
product_list_path = os.path.join(data_dir, product_list_fn)

out_fn = 'image_processed.out'
out_path = os.path.join(data_dir, out_fn)

# read in users list
with open (product_list_path, 'rb') as fp:
    product_list = pickle.load(fp)
product_list = set(product_list)

In [23]:
'B000FDXWFM' in product_list

False

In [26]:
df = {}
for d in readImageFeatures(image_features_path):
    asin, image_vector = d
    if asin in product_list:
        print(asin)
        df[asin] = image_vector

B000EPJL1A
B000IDC0K2
B000EHYG9K
B000HGC04G
B0007GIXSI
B000EWD6YG
B000BSJJ1M
B000FOUWNQ
B000E3XB2W
B0007SXHP0
B000E8T8SS
B0007M2X9M
B0009JFF7U
B0007LQKMO
B000EOOUY4
B000FDXH84
B000EI3W8U
B000ID37EA
B000EVLS4C
B000EONVZI
B000FOL7Q2
B000F3DYUU
B000F6SR0E
B000FZ33AE
B000BP8AY2
B000BEQLFS
B000F2P0ZS
B000F1UFZ4
B000FVIHY0
B000FV8P24
B000F6WK5M
B000BH2W9O
B0007U0GZM
B000GW81BM
B000FMPENQ
B0007RALGO
B000BD8FVC
B000FQ76IS
B000B8YH3W
B000EHLIHS
B000F8DBF8
B000BSJCT6
B000F2R5HO
B000FHC0Y2
B000BPCWEQ
B000EVS4YY
B000FVUZYA
B000799LQS
B000EVSXMC
B000EWHH7I
B000FIY08U
B0007L8BY4
B000F3RKJ6
B000FIQBNW
B000IHGJ50
B000GU781I
B000FUKGCW
B000AYWOZU
B000F9LRZI
B000BGTVDK
B000E8SWQM
B000EWI8IK
B000E9BXDU
B000FOI2TW
B000F8LQ0A
B000ETBYHK
B000EYRIGQ
B000FNGLSM
B000EI5QWA
B000I3DTCA
B000EU0UVA
B000FCT04U
B000BQN4IS
B000HGL5RE
B000F4TCDM
B000G0LL48
B000ARIT5Q
B000G61X5O
B000I94FRM
B000EN4CVQ
B000GP5F04
B0007L72NA
B000C1DXLU
B000FPIVLU
B000GSM4N2
B000AROO1Y
B000F7JCRA
B000FTHR7U
B000EZP1QE
B000EYPO7Q
B000BP4ZQY

B002IY4V9C
B002SN777G
B001IB4WH2
B002ROIHPC
B002ROGCRC
B001GUY5V8
B002R35IVY
B00154H1T8
B002RL8INQ
B002RWJGI6
B002FO6VTI
B002OV1VJW
B002RL9XQW
B003F9QBTW
B0030UF24G
B002RL9WEK
B002GHBTNC
B002TLTIS8
B002R5AFUG
B002RURFGS
B003FX68S2
B002HU629E
B002ECZ89U
B002S2F4F4
B002QEBMAK
B002JTE0W4
B002ZLN996
B002WBV37Y
B001H940ZO
B002KDZ8WK
B002TNRTSC
B002RWJDN4
B002P8CZ3A
B002SOZ5RY
B002WI8GG8
B002RWJD98
B002PZHXYE
B002KRU1IC
B002KDOJ8Y
B0010X1VJ0
B001GV22V2
B002Z58GQI
B00338OCCS
B002VX0GJY
B002RL8ICC
B001H9O71Q
B002P5J02M
B002K450T0
B002S4SNOQ
B002ZBJSIM
B002HSVPH0
B002WGJFJW
B002JGTMO8
B002SQFZ90
B0015L4XFQ
B002JTWSMS
B002ISTLKW
B002JTX8Y0
B002KETE24
B002RW08DS
B002WFN82S
B002RO8WM0
B002SCPH9W
B002KD9BMS
B002Q887BS
B002WSHXQ2
B001H5T7PQ
B002RCQ0SU
B002USJ1WI
B0039NLS3I
B002S43JK4
B002RWJHBM
B007K7LZJG
B008KNLZ3U
B007JLDDSO
B008I648Q0
B007JNWH8Y
B007JOO4YS
B008L3I9JC
B008J572MC
B008KQ0A7Y
B008DFVQFW
B008JHOT12
B008HO23OC
B008LRC4MQ
B008KN38IA
B008KNDTB6
B009A6PJKQ
B008KW62WA
B008IISC4M
B008D66RGO

B003WKSNKO
B004YF9K7G
B0041A1PZE
B005BDAR5E
B004YIZWWU
B0062KGAPQ
B0050SLTPC
B003VTZFNE
B005BOS4Z8
B004YNK9O6
B005DCVGWG
B006JG2IYA
B005FIX5FE
B005AZ7B4S
B006T3AUCU
B005D75PPK
B004D5PFGW
B005AS31O4
B003YTSXYO
B006TRANEQ
B00414O79Q
B005CSZ6MM
B0050SNCLQ
B005DHL0KY
B003X1KF62
B005ED1GX8
B005C95ND8
B005BZ3MEU
B005OPYH7I
B005CT2FWU
B004YW5406
B0058W1DH4
B004D5RFCE
B0062CWKCQ
B005CWJ76O
B004DI4KT2
B003WQMSOU
B003WM2N94
B005C2A4DO
B004Y9BG4C
B0053NBLFW
B005F7794S
B00419N288
B004DESA4C
B005ECB3PU
B005D0AE8K
B005B47AIU
B003X242PQ
B0054CFNJM
B003VQQVD0
B005D5LZZ6
B005F77Z9C
B005C6CVAE
B005D6P0N8
B005BZNDCG
B0053YJLL2
B00542NI58
B005BZNCKO
B003XU7H1O
B005EV1R8Y
B003VTZ0D4
B0050I568S
B003YWVPN2
B003WSTTM2
B004CM4A58
B006J73RAI
B005Z4R45K
B002LE8CPS
B005DWR4A4
B005BNP5WE
B0051SVVX6
B0051UWCH8
B006IJHK96
B0053TFCGK
B0041A3LJC
B005BRET3G
B005BON4K8
B003VQQVDK
B005FDOG1G
B005E2VS3W
B005DZJGI4
B0053VZUW4
B000YJIJI2
B000UHIMG2
B0014EHYZA
B001413EQ6
B000V5DK2O
B000WTMAP2
B000TMI07O
B000UN8MKM
B00115U2PG

B00B7IPT6Y
B00AWKYB1M
B00C9L6UO8
B009MJ58OI
B009LB2V3I
B00ATZ9IUY
B00B2REYQG
B00AUIIXSI
B00B332A24
B00B2XPOK0
B00B353NI2
B009NE78Z4
B00C6WI88Q
B00C9I4STA
B00B2G2GSU
B00B3ODHSE
B00ATM1M92
B003TY8LRI
B003U8HYIA
B00598NJ10
B005955ZN8
B0050K9MG8
B00569WY7M
B004ADIITI
B003VAZ6OG
B003ZM9TMA
B005EVGS2E
B003UD9ZOQ
B004ZEZGE2
B005A0B49Q
B00485MEH0
B005C3GYE6
B00568KKOW
B003WMOMZC
B005LN51JG
B0056EYB1E
B0057J3ZQK
B003UKH8HU
B005CDSMQE
B003TLY9UO
B00564BIJ2
B003T90WY8
B005AROCAW
B003U84YCO
B0049GEFU2
B0056VMBOG
B005BJFDL6
B003V1619E
B005BZNEKM
B0049CPQ36
B005CL2E2Y
B003TY9S2A
B0058YY202
B0056KLG52
B0050MM40C
B00486USL8
B005CSNBF6
B0058WT2XG
B003TY10AS
B005LT6UPY
B005CKM36C
B004921NKQ
B004ZL2O9U
B0058P5S9A
B005CLMJLU
B00568V8S4
B0049EGWWS
B004YU36ZS
B0056MPVZ6
B0056MHU82
B003UATR3I
B004YI7SNG
B0057XC2MO
B00573YYW0
B0057PTLIU
B005KDYBIO
B00567H67C
B0058LUHYA
B005OHRK2U
B0050C9ZMC
B004ZGLM5W
B005OU4Q9W
B0056PVM5G
B006K554BE
B0050G38L2
B00511F3BY
B0058N5TD2
B005AY01RI
B0050HGZC0
B004ZJ14NE
B0055LGG3Y

B008VO54H6
B00AA9K2GI
B00AAIC5S2
B00AAZ8OS0
B00BD7UVKI
B008U25LI6
B00AA48BZW
B008TYDEJS
B009Z8T3K6
B00A0LZZ9U
B008VPBXZ2
B00AAKLWGG
B00A6JH49A
B008UWXQWO
B00AAKLFHM
B00AUKGBE4
B00AA5LPT0
B008UCNCF0
B00AAIPVH4
B00A9GIFFC
B008VQ3A6Q
B00AB25PIE
B00A2BHC8A
B00A48RMKE
B00AAGBJGI
B008S4TSAI
B008TYVS6O
B00AA6CS86
B008V99350
B00AAFK5SW
B00A9S3OMY
B008UNROIA
B008VGQ4VO
B00AA5AWDU
B00AASENXC
B00A6UTJ3S
B008UY5WL0
B00A4N215U
B00AA6J1Z4
B008WV43PW
B00AAUMV0W
B009TLA7P4
B008UHK3KM
B008S55WJS
B008VHFNK6
B00A3OEW16
B00AAKF8O8
B00A3G461K
B00AB3CGF8
B008UE8YKG
B00AA2ZBT8
B00AAN0HQY
B00AAKLOBO
B00A9S3OOC
B008VNE54A
B008VMT2EE
B00A2G4ZN0
B003NZS37G
B003MVZ60I
B001K39980
B003NMW1HC
B003EE72C8
B003MATPQK
B003LJDF7W
B003TUMDWG
B003UDPU0Y
B003DW4A76
B003MGAZ1S
B003HH96IU
B003NVMDYY
B003KJOE0K
B002GIAUDG
B0028QWJTO
B003S5SOLG
B003DXI288
B003CQD10K
B003U8K66W
B003CFATR4
B003KWCBHA
B0041SN3E2
B001L6J3XC
B001MRYT3E
B002GHCZMG
B003S6AXVO
B003P2UMOE
B00295RBQ0
B002AUJHLG
B003LT8OPU
B003LZ6GL8
B003E47926
B003N1KYE0

...

B006P1JG3K
B006LQHT3I
B006R5KHSM
B00685U5GU
B007W7PJ78
B006P0JS34
B004J359Z0
B006MLKH4U
B006BCK7RM
B006LFNS7A
B005FVDS62
B006K552AC
B006QFZ2QA
B006K554UU
B006PL97C0
B00695JES4
B006Q6RAB4
B006PAAMA2
B006OE5JFW
B005X9XW8K
B006ISD0ZU
B004ILA5UC
B006WYVJ6W
B004J6GMLM
B00675W4W4
B006MVUQGY
B006LP0FXA
B006OS6UAG
B004IKAPOO
B006M4SIHU
B005Y4H3YI
B006OTWJVO
B007938DKA
B006K556QM
B006K1R3TY
B004I7XY9K
B006P60GJI
B006MX0WHU
B006QCQWP8
B006WWMZSK
B006M15D8U
B006P2SZ6S
B0069J1RLC
B006J70YRM
B007BGFNBM
B006MPRFJQ
B0067I54ZU
B006JPEDKI
B004I6MAV4
B0067TQQI8
B00816WDVO
B006REGE4O
B006M9ZVXY
B004ISXB4C
B006OAO894
B00JOI6FZ8
B00JQ4HZPO
B00JRULWM4
B00JXLGE6Q
B00JG59X9O
B00JEOC1LO
B00JWV1LP6
B00JJWH3PU
B00K2MAU5Q
B00JFRANY2
B00JOXN8YY
B00KFAGCUM
B00JVVU0SQ
B00JCSLDAW
B00JST4VHW
B00JEU3ZA4
B00JKUPRF4
B00JJED87A
B00K3K8TKA
B00JGL37FO
B00JKM0ACW
B00JXLGECA
B000069E1H
B00006HCIX
B00005R098
B000068UXB
B00005NHHA
B0000630TK
B000067RJB
B00006B8EC
B00006B9W3
B00005Z24Y
B00005V619
B00005NBIT
B000068ODT
B00005AXHW

B009PK9UW0
B009NL265O
B009NE5Y9G
B00A3YN0Z0
B00A3HYETI
B00A2BVAN8
B00BBFZBHU
B00A2XIIVI
B00BNQ1GSK
B008PT4GGC
B00A42Y0PA
B00A3TCWSG
B008PF9454
B009UZBLSG
B009USAJCC
B008PQAB3C
B00ADC5SPW
B00BBGIQ62
B009NXOKOM
B009UQFL7W
B008PUPBF6
B00A3MY7L8
B00A3NIGQ4
B009W34X1I
B008P5LFEC
B008OZXKFA
B00A87HIQ4
B009T3LM1K
B008Q3CCZE
B00A4EAJAI
B00A627JOC
B00A5BEJBA
B00A45JEX0
B00A35X6NU
B00A2XII9A
B00A3TLMV4
B00A6HYP1W
B00A3VOA3O
B00A3EO4C8
B00A3WM3W8
B00A3NIFXI
B009N8JL9Q
B008OCHYO6
B00A3I34EI
B00A6HYK0I
B00A40EVCO
B00A42X2KY
B00AOAS1S4
B008PQAGM8
B00A82MU6W
B00A3FWBFE
B00A412X6E
B00A3R7666
B00A4C9J1A
B00A8FCX8E
B008QJXPGS
B008PO5CSI
B009XPQE0S
B008QUVSCA
B00A3M41EG
B00A404PMK
B009UHC5KC
B008PRPKIC
B00A3U7KDC
B00A47AWBQ
B00A659J88
B00CD0J4BG
B008GZU8AI
B008GWPC1Q
B007R6HTTA
B008I6RVZU
B007HIR3MG
B008I91MRU
B008GW6OBI
B008BMWPKW
B008I5PK22
B007R9CRGW
B007H41GFU
B008HCX6S6
B008HNP6W4
B008GAMNBK
B008B09LRY
B007H4QJA2
B0096TTOHG
B008GYTWQA
B008FLOWAU
B007F9XHAY
B008GVETNE
B008HK4ZAG
B007HD497I
B008OCNXDW

B0034G4YQS
B0033WIFMC
B002XDQAA6
B002ZLOR38
B002ZV51DI
B002ROCSC0
B001MS2UJ8
B002WVMHDI
B0031SEMBQ
B001AO10YU
B003H0TIVC
B001MQ8BL6
B002YK9SIE
B002RX43RE
B002Z3H36O
B002R5AM7C
B0030MHJ04
B0034XS250
B0030B6F1O
B002PJESBG
B0037KLEWS
B001NRUZMW
B0035CVDX8
B002QQ3VE8
B002NP8XJ0
B00307EL8M
B002YTGBKI
B003AOAXL4
B002N2LCQE
B002PY7LME
B0033RH30C
B002RWJDAC
B001LF40KE
B001MQ33AU
B00158QASW
B002QQ8IO6
B0030T17GO
B000PO045W
B001L4A76G
B0037AGZIG
B002R96LDC
B00145EVJ6
B002YXS8E6
B0035AMT8I
B001NHP96Y
B0031RGGBG
B0037FSYQC
B0037TEO3K
B002MZUNXU
B002ZFVS9A
B002S53LJ2
B002ZAVVHY
B002X46V36
B0033Y89K8
B002NWTEUK
B002OEBMS4
B00355IJU0
B001L2ZSSK
B002MZZR6I
B002Z3LQVC
B002POMZWU
B002WGAHYY
B001AN16QI
B003053L3U
B002WN21IM
B001NLX1LK
B0057PLN9K
B003ZG9T62
B00551931Q
B002QEBM96
B005HXE9WU
B0062HBRUM
B005HKIDF2
B00597C3T0
B0040Q4TLQ
B0040720NO
B004J6GNUC
B00630WKGI
B0065V3FHS
B004ILIJIC
B0052SCU8U
B003ZMTPG0
B00452YFZU
B0065SY3IG
B005FFRB8O
B004I1FBFQ
B005I0JYUY
B005FDJC6K
B005FY2DZG
B005ES6E6W
B0052QU322

B00005111L
B00004TY7O
B00009VQJ7
B0001G6U7A
B0001XGQSG
B00006HMQZ
B0001JSSLI
B0001GDEE2
B00006B9CN
B0001U5M48
B000117D9O
B0000AERK6
B00012O6RK
B00009K7E3
B00017LTFC
B0001ML86M
B000068O3S
B00006B80L
B0001N6LCM
B00004ZCKV
B00006HYYH
B00028CAJE
B0000633EM
B00006JPFC
B000068O3T
B0000YD2JC
B00006B8JX
B000234VZK
B00009VS6P
B000271OWO
B0001VNN6G
B00006345I
B00004ZCA0
B000066AX5
B0001N3ZRQ
B00027IOI6
B00009R6WU
B0002EHOSU
B00019EYVG
B0002D6Q0I
B00009WO2M
B000138VH0
B00009WBYJ
B000165C10
B00004SU73
B000167KPQ
B000063CUL
B00006HMQY
B0002BESFA
B0001G6U3Y
B0000DBMHR
B0001CKX2M
B0000X0VCY
B0001YFX7U
B000059TF8
B0001654WC
B0000AQNX2
B00005M1UY
B004RLOPE0
B004MFFD2Y
B004PA874M
B0053F482I
B000FBF400
B005EP3EH2
B000GF2J76
B004VPXLDI
B000MDY7K4
B000H9AZIQ
B000QJ1554
B004N6259G
B004U6JNX0
B005EZKFBU
B000J01HXU
B000FLNU4M
B000M9MKES
B000PCF8WS
B000EZUDQ2
B004QZFOF6
B000MUXJCO
B000QJ3IBS
B000JC76B0
B000I97G4Q
B000KFYD9E
B000OBJW22
B004QF0VE0
B000PO4S54
B004X1SFTA
B005EM4PXC
B004MME698
B000OTLU2O
B005E7TE9W

B004J2NVDI
B004TB75L8
B004QMFBPW
B004GCJEZU
B003I4FD9I
B004S958VU
B004F0D2A6
B004PVW1DE
B003IZFCFW
B004OZ6BQ4
B004OBZ088
B004QRUM92
B004I8U226
B0039GGZ3I
B004RDLQV8
B004R71D0I
B0057XBYVY
B004SY9OVA
B004RO3QLU
B003AXW2YQ
B0039BT2DI
B004Q7LGZG
B004IVAPM0
B004PBE9KW
B004J5CM9O
B004PADOPO
B004RL6QUQ
B004S862FW
B004IGVT8Y
B0036NWI94
B0051XJXJU
B004I44EVA
B00522X8B4
B004UE1Q7S
B004T9RR6I
B003HIXOPU
B004VY47I2
B004SGSH4S
B004ZEZBEC
B003HK2LRK
B0050751YS
B004QX2FU0
B0036WT4JW
B004SP1390
B004S7W1RQ
B004R7CIBQ
B0057LX9H8
B004P1ITUI
B0036QL1JY
B004TPG3ZS
B004I8W55S
B004JRY402
B004U4WPZA
B004PJTV5W
B003IT6PHC
B0038P1TP4
B004SD1ZPY
B005C31HC0
B000056KYO
B004SBBDD0
B004VMM718
B004RCNKC2
B00005MNSR
B004QF0VD6
B004S95AZO
B004WP92NK
B004X6ABBA
B004TFTE0O
B004QD002A
B004SJY2LC
B004TTUP8U
B004QJ3ML0
B005GI18EI
B004S8A6SQ
B003WGNSTE
B004S330V6
B004SKBIH2
B004RBT9XW
B004PVW1DY
B004SP0NMI
B005GFWQO2
B004P8L9BM
B004SGKD2C
B004S52MGS
B004SKBHB4
B004SBK7TQ
B004QK7HI8
B005BNYV1K
B004R73YPU
B004SAR998
B004R5SLJG

In [3]:
with open(os.path.join('../data', 'image_processed.out'), 'rb') as fp:
    d = pickle.load(fp)

In [4]:
len(d)

62503

In [76]:
with open (os.path.join('../data', 'products.out'), 'rb') as fp:
    product_list = pickle.load(fp)

In [77]:
len(product_list)

62983

In [78]:
sp = set(product_list)
sd = set(d.keys())

In [79]:
len(sp)

62983

In [82]:
len(sp.intersection(sd))

62503

In [68]:
product_list = set(product_list)
len(product_list)

62983

In [10]:
no_img = []
for p in product_list:
    if p not in d.keys():
        no_img.append(p)

In [34]:
for i in prod:
    if i not in product_list:
        print(i)

In [40]:
len(reviews[reviews['asin'].isin(d.keys())].asin.unique())

62503